# Setup Environment
References:  


In [1]:
!pip install instagrapi 


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# 패키지/모듈 로딩

# Library Test::instagrapi 

In [5]:
from instagrapi import Client
Client 

instagrapi.Client

In [8]:
client = Client()
client 

In [7]:
v = client.login('ahoragabriele@gmail.com', '!5272doubleO')
print(v)

ProxyAddressIsBlocked: Instagram has blocked your IP address, use a quality proxy provider (not free, not shared)

In [9]:
before_ip = client._send_public_request("https://api.ipify.org/")
before_ip

ClientConnectionError: ConnectionError HTTPSConnectionPool(host='api.ipify.org', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000187486FB820>: Failed to resolve 'api.ipify.org' ([Errno 11001] getaddrinfo failed)"))

# Selenium Driver 를 이용해서 다운로드 
SSS Instagram Web Service  
https://sssinstagram.com/  

In [32]:
import re 
from time import sleep

from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
driver = webdriver.Chrome()
driver 

<selenium.webdriver.chrome.webdriver.WebDriver (session="54a0da319284fe59119521a2f1f89925")>

In [96]:
driver.get("https://sssinstagram.com/")

In [97]:
def select_contents_type(type='Video'):
    app = driver.find_element(By.ID, 'app')
    nav = app.find_element(By.CLASS_NAME, "pageTunerWrapper")
    # print(nav)

    type = type.capitalize()
    # print('type:', type)
    a = nav.find_element(By.LINK_TEXT, type)
    # print(a)
    a.click()

select_contents_type('video')

### 링크주소 입력

In [98]:
def input_link(url):
    input_box = driver.find_element(By.ID, 'input')
    # print(input_box) 

    input_box.send_keys(url)
    input_box.send_keys(Keys.RETURN)

input_link("https://www.instagram.com/p/C-hts7migZz/?utm_source=ig_web_copy_link") 

In [123]:
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=2)
import os 
from bs4 import BeautifulSoup
import requests 
from moviepy.editor import VideoFileClip, concatenate_videoclips

def get_download_urls():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    app = soup.find(id='app')
    result = app.find(class_="search-result")
    _list = result.find('ul', class_="output-list__list")
    # print(_list.prettify())
    urls = []
    for li in _list.find_all('li'):
        d = li.find('a').attrs
        urls.append(d['href'])
    return urls 


def download_file(url, local_filename):
    # Send a GET request to the URL
    with requests.get(url, stream=True) as response:
        response.raise_for_status()  # Check for HTTP request errors
        # Open a local file with write-binary mode
        with open(local_filename, 'wb') as file:
            # Write the content of the response in chunks
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
    print(f"File downloaded: {local_filename}")


def download(delay):
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "search-result"))
        )
    finally:
        print('results', results)

        # ul = results.find_element(By.CLASS_NAME, 'output-list__list')
        # print(ul)
        try:
            ul = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "output-list__list"))
            )
        finally:
            print('ul', ul)

            # 방법.1
            # for i, li in enumerate(ul.find_elements(By.TAG_NAME, 'li'), start=1):
            #     print([i, li])
            #     li.find_element(By.LINK_TEXT, 'Download').click()
            #     sleep(delay)

            # 방법.2
            urls = get_download_urls()
            filepaths = []
            for i, url in enumerate(urls, start=1):
                print([i, url])
                _dir = 'C:\\Users\\innovata\\Downloads'
                filepath = os.path.join(_dir, str(i).zfill(2)+'.mp4')
                filepaths.append(filepath)
                download_file(url, filepath)

            print('다운로드 완료')
            
            # 동영상 파일 병합
            clips = []
            for filepath in filepaths:
                clips.append(VideoFileClip(filepath))
            final_clip = concatenate_videoclips(clips)
            out_file = os.path.join(_dir, 'output.mp4')
            final_clip.write_videofile(out_file)

            # 다운로드한 분할 영상은 삭제 
            for filepath in filepaths:
                os.remove(filepath)

            print('DONE.')


# urls = get_download_urls()
# download(3)

In [105]:
from urllib.parse import urlparse
o = urlparse(urls[0])

In [109]:
o.path 

'/get'

In [122]:
def download_insta(url, type='video'):
    driver.get("https://sssinstagram.com/")
    select_contents_type(type)
    input_link(url) 
    download(delay=2)

download_insta("https://www.instagram.com/p/C-hts7migZz/?utm_source=ig_web_copy_link", type='video')

results <selenium.webdriver.remote.webelement.WebElement (session="54a0da319284fe59119521a2f1f89925", element="f.684996BD7082411008F6790B683F615A.d.A0E06F0F6945CBD933DE3C852BCAC7B2.e.37418")>
ul <selenium.webdriver.remote.webelement.WebElement (session="54a0da319284fe59119521a2f1f89925", element="f.684996BD7082411008F6790B683F615A.d.A0E06F0F6945CBD933DE3C852BCAC7B2.e.37616")>
[1, 'https://media.sssinstagram.com/get?__sig=6EwEgEVF8qUw7_UMaKUs8Q&__expires=1723388632&uri=https%3A%2F%2Finstagram.ffjr1-4.fna.fbcdn.net%2Fo1%2Fv%2Ft16%2Ff2%2Fm69%2FAn_3TQZyCF2zEgJCgW26RHOHn7_mZI4o5T5UFn3ak7SJIFIfqqmyUXVPfNOwrz9bLgAixXlBWj_7KkvKvFD02NH_.mp4%3Fefg%3DeyJxZV9ncm91cHMiOiJbXCJpZ193ZWJfZGVsaXZlcnlfdnRzX290ZlwiXSIsInZlbmNvZGVfdGFnIjoidnRzX3ZvZF91cmxnZW4uY2Fyb3VzZWxfaXRlbS5jMi4xMDgwLmJhc2VsaW5lIn0%26_nc_ht%3Dinstagram.ffjr1-4.fna.fbcdn.net%26_nc_cat%3D106%26vs%3D1024068939307718_2419050817%26_nc_vs%3DHBksFQIYOnBhc3N0aHJvdWdoX2V2ZXJzdG9yZS9HUFVPRlJNdjJvS2lmMm9CQUxHdlJ1U2RBZUJSYnBSMUFBQUYVAALIAQAVAhg6cGF

MoviePy - Done.
Moviepy - Writing video C:\Users\innovata\Downloads\output.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\innovata\Downloads\output.mp4
DONE.
